In [1]:
from __future__ import division

import sys
import os
import time
import math
# import ipdb
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('residual_net_n', 5, '')
tf.app.flags.DEFINE_string('train_tf_path', './data/train.tf', '')
tf.app.flags.DEFINE_string('val_tf_path', './data/test.tf', '')
tf.app.flags.DEFINE_integer('train_batch_size', 128, '')
tf.app.flags.DEFINE_integer('val_batch_size', 100, '')
tf.app.flags.DEFINE_float('weight_decay', 0.0005, 'Weight decay')
tf.app.flags.DEFINE_integer('summary_interval', 100, 'Interval for summary.')
tf.app.flags.DEFINE_integer('val_interval', 1000, 'Interval for evaluation.')
tf.app.flags.DEFINE_integer('max_steps', 80000, 'Maximum number of iterations.')
tf.app.flags.DEFINE_integer('save_interval', 5000, '')
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [3]:
def one_hot_embedding(label, n_classes):
  """
  One-hot embedding
  Args:
    label: int32 tensor [B]
    n_classes: int32, number of classes
  Return:
    embedding: tensor [B x n_classes]
  """
  embedding_params = np.eye(n_classes, dtype=np.float32)
  with tf.device('/cpu:0'):
    params = tf.constant(embedding_params)
    embedding = tf.gather(params, label)
  return embedding

In [4]:
def conv2d(x, n_in, n_out, k, s, p='SAME', bias=False, scope='conv'):
  with tf.variable_scope(scope):
    kernel = tf.Variable(
      tf.truncated_normal([k, k, n_in, n_out],
        stddev=math.sqrt(2/(k*k*n_in))),
      name='weight')
    tf.add_to_collection('weights', kernel)
    conv = tf.nn.conv2d(x, kernel, [1,s,s,1], padding=p)
    if bias:
      bias = tf.get_variable('bias', [n_out], initializer=tf.constant_initializer(0.0))
      tf.add_to_collection('biases', bias)
      conv = tf.nn.bias_add(conv, bias)
  return conv

In [5]:
def batch_norm(x, n_out, phase_train, scope='bn', affine=True):
  """
  Batch normalization on convolutional maps.
  Args:
    x: Tensor, 4D BHWD input maps
    n_out: integer, depth of input maps
    phase_train: boolean tf.Variable, true indicates training phase
    scope: string, variable scope
    affine: whether to affine-transform outputs
  Return:
    normed: batch-normalized maps
  """
  with tf.variable_scope(scope):
    beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
      name='beta', trainable=True)
    gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
      name='gamma', trainable=affine)
    tf.add_to_collection('biases', beta)
    tf.add_to_collection('weights', gamma)

    batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
    ema = tf.train.ExponentialMovingAverage(decay=0.99)

    def mean_var_with_update():
      ema_apply_op = ema.apply([batch_mean, batch_var])
      with tf.control_dependencies([ema_apply_op]):
        return tf.identity(batch_mean), tf.identity(batch_var)
    mean, var = control_flow_ops.cond(phase_train,
      mean_var_with_update,
      lambda: (ema.average(batch_mean), ema.average(batch_var)))

    normed = tf.nn.batch_norm_with_global_normalization(x, mean, var, 
      beta, gamma, 1e-3, affine)
  return normed

In [6]:
def residual_block(x, n_in, n_out, subsample, phase_train, scope='res_block'):
  with tf.variable_scope(scope):
    if subsample:
      y = conv2d(x, n_in, n_out, 3, 2, 'SAME', False, scope='conv_1')
      shortcut = conv2d(x, n_in, n_out, 3, 2, 'SAME',
                False, scope='shortcut')
    else:
      y = conv2d(x, n_in, n_out, 3, 1, 'SAME', False, scope='conv_1')
      shortcut = tf.identity(x, name='shortcut')
    y = batch_norm(y, n_out, phase_train, scope='bn_1')
    y = tf.nn.relu(y, name='relu_1')
    y = conv2d(y, n_out, n_out, 3, 1, 'SAME', True, scope='conv_2')
    y = batch_norm(y, n_out, phase_train, scope='bn_2')
    y = y + shortcut
    y = tf.nn.relu(y, name='relu_2')
  return y

In [7]:
def residual_group(x, n_in, n_out, n, first_subsample, phase_train, scope='res_group'):
  with tf.variable_scope(scope):
    y = residual_block(x, n_in, n_out, first_subsample, phase_train, scope='block_1')
    for i in range(n - 1):
      y = residual_block(y, n_out, n_out, False, phase_train, scope='block_%d' % (i + 2))
  return y

In [8]:
def residual_net(x, n, n_classes, phase_train, scope='res_net'):
  with tf.variable_scope(scope):
    y = conv2d(x, 3, 16, 3, 1, 'SAME', False, scope='conv_init')
    y = batch_norm(y, 16, phase_train, scope='bn_init')
    y = tf.nn.relu(y, name='relu_init')
    y = residual_group(y, 16, 16, n, False, phase_train, scope='group_1')
    y = residual_group(y, 16, 32, n, True, phase_train, scope='group_2')
    y = residual_group(y, 32, 64, n, True, phase_train, scope='group_3')
#     y = conv2d(y, 64, n_classes, 1, 1, 'SAME', True, scope='conv_last')
    y = tf.nn.avg_pool(y, [1, 8, 8, 1], [1, 1, 1, 1], 'VALID', name='avg_pool')
    y = tf.reshape(y, [-1, 64])
    w = tf.get_variable(name='weight_fc', shape=[64, n_classes], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    tf.add_to_collection('weights', w)
    b = tf.get_variable(name='weight_biase', shape=[n_classes], initializer=tf.constant_initializer(0))
    tf.add_to_collection('last_biases', b)
    y = tf.matmul(y, w) + b
#     y = tf.squeeze(y, squeeze_dims=[1, 2])
  return y


In [9]:
def _loss(logits, labels, scope='loss'):
  with tf.variable_scope(scope):
    # entropy loss
    targets = one_hot_embedding(labels, 10)
    entropy_loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets),
      name='entropy_loss')
    tf.add_to_collection('losses', entropy_loss)
    # weight l2 decay loss
    weight_l2_losses = [tf.nn.l2_loss(o) for o in tf.get_collection('weights')]
    weight_decay_loss = FLAGS.weight_decay*tf.add_n(weight_l2_losses)
    tf.add_to_collection('losses', weight_decay_loss)
  # for var in tf.get_collection('losses'):
    # tf.scalar_summary('losses/' + var.op.name, var)
  # total loss
  return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [10]:
def _accuracy(logits, gt_label, scope='accuracy'):
  with tf.variable_scope(scope):
    pred_label = tf.argmax(logits, 1)
    acc = 1.0 - tf.nn.zero_fraction(
      tf.cast(tf.equal(pred_label, gt_label), tf.int32))
  return acc

In [11]:
def _train_op(loss, global_step, learning_rate):
  params = tf.trainable_variables()
  gradients = tf.gradients(loss, params, name='gradients')
  optim = tf.train.MomentumOptimizer(learning_rate, 0.9)
  update = optim.apply_gradients(zip(gradients, params))
  with tf.control_dependencies([update]):
    train_op = tf.no_op(name='train_op')
  return train_op

In [12]:
def cifar10_input_stream(records_path):
  reader = tf.TFRecordReader()
  filename_queue = tf.train.string_input_producer([records_path], None)
  _, record_value = reader.read(filename_queue)
  features = tf.parse_single_example(record_value,
    {
      'image_raw': tf.FixedLenFeature([], tf.string),
      'label': tf.FixedLenFeature([], tf.int64),
    })
  image = tf.decode_raw(features['image_raw'], tf.uint8)
  image = tf.reshape(image, [32,32,3])
  image = tf.cast(image, tf.float32)
  label = tf.cast(features['label'], tf.int64)
  return image, label

In [13]:
def normalize_image(image):
  # meanstd = joblib.load(FLAGS.mean_std_path)
  # mean, std = meanstd['mean'], meanstd['std']
  mean = [ 125.30690002,122.95014954,113.86599731]
  std = [ 62.9932518,62.08860397,66.70500946]
  normed_image = (image - mean) / std
  return normed_image

In [14]:
def random_distort_image(image):
  distorted_image = image
  distorted_image = tf.image.pad_to_bounding_box(
    image, 4, 4, 40, 40)  # pad 4 pixels to each side
  distorted_image = tf.random_crop(distorted_image, [32, 32, 3])
  distorted_image = tf.image.random_flip_left_right(distorted_image)
  return distorted_image


In [15]:
def make_train_batch(train_records_path, batch_size):
  with tf.variable_scope('train_batch'):
    with tf.device('/cpu:0'):
      train_image, train_label = cifar10_input_stream(train_records_path)
      train_image = normalize_image(train_image)
      train_image = random_distort_image(train_image)
      train_image_batch, train_label_batch = tf.train.shuffle_batch(
        [train_image, train_label], batch_size=batch_size, num_threads=4,
        capacity=50000,
        min_after_dequeue=1000)
  return train_image_batch, train_label_batch

In [16]:
def make_validation_batch(test_records_path, batch_size):
  with tf.variable_scope('evaluate_batch'):
    with tf.device('/cpu:0'):
      test_image, test_label = cifar10_input_stream(test_records_path)
      test_image = normalize_image(test_image)
      test_image_batch, test_label_batch = tf.train.batch(
        [test_image, test_label], batch_size=batch_size, num_threads=1,
        capacity=10000)
  return test_image_batch, test_label_batch

In [17]:
phase_train = tf.placeholder(tf.bool, name='phase_train')
learning_rate = tf.placeholder(tf.float32, name='learning_rate')

global_step = tf.Variable(0, trainable=False, name='global_step')


train_image_batch, train_label_batch = make_train_batch(FLAGS.train_tf_path, FLAGS.train_batch_size)
val_image_batch, val_label_batch = make_validation_batch(FLAGS.val_tf_path, FLAGS.val_batch_size)

image_batch, label_batch = control_flow_ops.cond(phase_train,lambda: (train_image_batch, train_label_batch),lambda: (val_image_batch, val_label_batch))


logits = residual_net(image_batch, FLAGS.residual_net_n, 10, phase_train)


loss = _loss(logits, label_batch)
accuracy = _accuracy(logits, label_batch)

# train one step


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [18]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
print('Initializing...')
saver = tf.train.Saver()
saver.restore(sess,'./full_precision/res32/model/res.ckpt')

tf.train.start_queue_runners(sess=sess)

Initializing...
INFO:tensorflow:Restoring parameters from ./full_precision/res32/model/res.ckpt


[<Thread(QueueRunnerThread-train_batch/input_producer-train_batch/input_producer/input_producer_EnqueueMany, started daemon 140035923896064)>,
 <Thread(QueueRunnerThread-train_batch/shuffle_batch/random_shuffle_queue-train_batch/shuffle_batch/random_shuffle_queue_enqueue, started daemon 140035932288768)>,
 <Thread(QueueRunnerThread-train_batch/shuffle_batch/random_shuffle_queue-train_batch/shuffle_batch/random_shuffle_queue_enqueue, started daemon 140036551620352)>,
 <Thread(QueueRunnerThread-train_batch/shuffle_batch/random_shuffle_queue-train_batch/shuffle_batch/random_shuffle_queue_enqueue, started daemon 140047894419200)>,
 <Thread(QueueRunnerThread-train_batch/shuffle_batch/random_shuffle_queue-train_batch/shuffle_batch/random_shuffle_queue_enqueue, started daemon 140035915503360)>,
 <Thread(QueueRunnerThread-evaluate_batch/input_producer-evaluate_batch/input_producer/input_producer_EnqueueMany, started daemon 140035907110656)>,
 <Thread(QueueRunnerThread-evaluate_batch/batch/fifo

In [19]:

# validation

print('Evaluating...')
n_val_samples = 10000
val_batch_size = FLAGS.val_batch_size
n_val_batch = int(n_val_samples / val_batch_size)
val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
val_labels = np.zeros((n_val_samples), dtype=np.int64)
val_losses = []
for i in range(n_val_batch):
  fetches = [logits, label_batch, loss]
  session_outputs = sess.run(
    fetches, {phase_train.name: False})
  val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
  val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
  val_losses.append(session_outputs[2])
pred_labels = np.argmax(val_logits, axis=1)
val_accuracy = np.count_nonzero(
  pred_labels == val_labels) / n_val_samples
val_loss = float(np.mean(np.asarray(val_losses)))
print('Test accuracy = %f' % val_accuracy)

Evaluating...
Test accuracy = 0.924600


In [20]:
def apply_prune_on_grads(grads_and_vars, dict_nzidx):
    for key, nzidx in dict_nzidx.items():
        count = 0
        for grad, var in grads_and_vars:
            if var.name == key:
                nzidx_obj = tf.cast(tf.constant(sess.run(dict_nzidx[key])), tf.float32)
                grads_and_vars[count] = (tf.multiply(nzidx_obj, grad), var)
            count += 1
    return grads_and_vars

def apply_inq(weights, inq_dict, var_name, prune_rate):  
    for target in var_name:
        wl = target
        bit = 16

        weight_obj = weights[wl]
        weight_arr = sess.run(weight_obj)

        weight_rest = np.reshape(weight_arr, [-1])
        dic_tem = np.reshape(sess.run(inq_dict[wl]), [-1])
        idx_rest = np.flip(np.argsort(abs(np.reshape(weight_rest, [-1]))), 0)
        num_prune = int(len(weight_rest) * prune_rate)
        weight_toINQ = weight_rest[idx_rest[:num_prune]]

        n1 = (np.floor(np.log2(max(abs(np.reshape(weight_arr, [-1]))) * 4 / 3)))
        n2 = n1 + 1 - bit / 4
        upper_bound = 2 ** (np.floor(np.log2(max(abs(np.reshape(weight_arr, [-1]))) * 4 / 3)))
        lower_bound = 2 ** (n1 + 1 - bit / 4)

        weight_toINQ[abs(weight_toINQ) < lower_bound] = 0
        weight_toINQ[weight_toINQ != 0] = 2 ** (np.floor(np.log2(abs(weight_toINQ[weight_toINQ != 0] * 4 / 3)))) * np.sign(weight_toINQ[weight_toINQ != 0])

        weight_rest[idx_rest[:num_prune]] = weight_toINQ
        weight_arr = np.reshape(weight_rest, np.shape(weight_arr))
        dic_tem[idx_rest[:num_prune]] = np.zeros_like(dic_tem[idx_rest[:num_prune]])
        inq_dict[wl] = tf.cast(np.reshape(dic_tem, np.shape(sess.run(inq_dict[wl]))), tf.float32)
        sess.run(weights[wl].assign(weight_arr))
    return inq_dict

# 开始压缩

第一轮  量化
prune_rate =0.5

In [21]:
print('prune前的准确率')
n_val_samples = 10000
val_batch_size = FLAGS.val_batch_size
n_val_batch = int(n_val_samples / val_batch_size)
val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
val_labels = np.zeros((n_val_samples), dtype=np.int64)
val_losses = []
for i in range(n_val_batch):
    fetches = [logits, label_batch, loss]
    session_outputs = sess.run(fetches, {phase_train.name: False})
    val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
    val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
val_losses.append(session_outputs[2])
pred_labels = np.argmax(val_logits, axis=1)
val_accuracy = np.count_nonzero(pred_labels == val_labels) / n_val_samples
val_loss = float(np.mean(np.asarray(val_losses)))
print('Test accuracy = %f' % val_accuracy)

prune前的准确率
Test accuracy = 0.924600


In [22]:
print('prune_rate = 0时，可视化部分参数：res_net/conv_last/bias:0')
print(sess.run(tf.get_collection('last_biases')[0]))

prune_rate = 0时，可视化部分参数：res_net/conv_last/bias:0
[ 0.02709051 -0.08279994  0.01822233  0.11905745 -0.00915685 -0.03560071
  0.01750009  0.03826123  0.00807592 -0.10065438]


In [23]:
para_dict = {}
one_dict = {}
var_name = []
for k in tf.trainable_variables():
    para_dict[k.name] = k
    one_dict[k.name] =tf.ones_like(k)
    var_name.append(k.name)
prune_dict=apply_inq(para_dict,one_dict,var_name,0.5)
trainer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = trainer.compute_gradients(loss)
grads_and_vars = apply_prune_on_grads(grads_and_vars, prune_dict)
train_step = trainer.apply_gradients(grads_and_vars)

In [24]:
curr_lr = 0
for step in range(40000):
  if step <= 48000:
    _lr = 1e-2
  else:
    _lr = 1e-3
  if curr_lr != _lr:
    curr_lr = _lr
    print('Learning rate set to %f' % curr_lr)

  # train
  fetches = [train_step, loss]
  if step > 0 and step % FLAGS.summary_interval == 0:
    fetches += [accuracy]
  sess_outputs = sess.run(fetches, {phase_train.name: True, learning_rate.name: curr_lr})


  if step > 0 and step % FLAGS.summary_interval == 0:
    train_loss_value, train_acc_value= sess_outputs[1:]
    print('[%s] Iteration %d, train loss = %f, train accuracy = %f' %
        (datetime.now(), step, train_loss_value, train_acc_value))

  # validation
  if step > 0 and step % FLAGS.val_interval == 0:
    print('Evaluating...')
    n_val_samples = 10000
    val_batch_size = FLAGS.val_batch_size
    n_val_batch = int(n_val_samples / val_batch_size)
    val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
    val_labels = np.zeros((n_val_samples), dtype=np.int64)
    val_losses = []
    for i in range(n_val_batch):
      fetches = [logits, label_batch, loss]
      session_outputs = sess.run(
        fetches, {phase_train.name: False})
      val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
      val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
      val_losses.append(session_outputs[2])
    pred_labels = np.argmax(val_logits, axis=1)
    val_accuracy = np.count_nonzero(
      pred_labels == val_labels) / n_val_samples
    val_loss = float(np.mean(np.asarray(val_losses)))
    print('Test accuracy = %f' % val_accuracy)

Learning rate set to 0.010000
[2018-07-16 16:40:26.762136] Iteration 100, train loss = 0.151927, train accuracy = 1.000000
[2018-07-16 16:40:32.397298] Iteration 200, train loss = 0.177482, train accuracy = 0.984375
[2018-07-16 16:40:38.045405] Iteration 300, train loss = 0.157973, train accuracy = 0.992188
[2018-07-16 16:40:43.706615] Iteration 400, train loss = 0.163516, train accuracy = 0.984375
[2018-07-16 16:40:49.388834] Iteration 500, train loss = 0.180150, train accuracy = 0.976562
[2018-07-16 16:40:55.050119] Iteration 600, train loss = 0.205976, train accuracy = 0.968750
[2018-07-16 16:41:00.692573] Iteration 700, train loss = 0.164587, train accuracy = 0.992188
[2018-07-16 16:41:06.330189] Iteration 800, train loss = 0.156320, train accuracy = 0.992188
[2018-07-16 16:41:11.973087] Iteration 900, train loss = 0.209756, train accuracy = 0.976562
[2018-07-16 16:41:17.634695] Iteration 1000, train loss = 0.142290, train accuracy = 1.000000
Evaluating...
Test accuracy = 0.920500


第二轮 量化
prune_rate = 0.75

In [25]:
print('prune前的准确率')
n_val_samples = 10000
val_batch_size = FLAGS.val_batch_size
n_val_batch = int(n_val_samples / val_batch_size)
val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
val_labels = np.zeros((n_val_samples), dtype=np.int64)
val_losses = []
for i in range(n_val_batch):
    fetches = [logits, label_batch, loss]
    session_outputs = sess.run(fetches, {phase_train.name: False})
    val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
    val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
val_losses.append(session_outputs[2])
pred_labels = np.argmax(val_logits, axis=1)
val_accuracy = np.count_nonzero(pred_labels == val_labels) / n_val_samples
val_loss = float(np.mean(np.asarray(val_losses)))
print('Test accuracy = %f' % val_accuracy)

prune前的准确率
Test accuracy = 0.923200


In [26]:
print('prune_rate =0.5时，可视化部分参数：res_net/conv_last/bias:0')
print(sess.run(tf.get_collection('last_biases')[0]))

prune_rate =0.5时，可视化部分参数：res_net/conv_last/bias:0
[ 0.04147975 -0.0625     -0.01364486  0.125       0.01460984 -0.03125
  0.02252148  0.03125     0.01663994 -0.125     ]


In [27]:
para_dict = {}
one_dict = {}
var_name = []
for k in tf.trainable_variables():
    para_dict[k.name] = k
    one_dict[k.name] =tf.ones_like(k)
    var_name.append(k.name)
prune_dict=apply_inq(para_dict,one_dict,var_name,0.75)
trainer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = trainer.compute_gradients(loss)
grads_and_vars = apply_prune_on_grads(grads_and_vars, prune_dict)
train_step = trainer.apply_gradients(grads_and_vars)

In [28]:
for step in range(FLAGS.max_steps):
  if step <= 48000:
    _lr = 1e-2
  else:
    _lr = 1e-3
  if curr_lr != _lr:
    curr_lr = _lr
    print('Learning rate set to %f' % curr_lr)

  # train
  fetches = [train_step, loss]
  if step > 0 and step % FLAGS.summary_interval == 0:
    fetches += [accuracy]
  sess_outputs = sess.run(fetches, {phase_train.name: True, learning_rate.name: curr_lr})


  if step > 0 and step % FLAGS.summary_interval == 0:
    train_loss_value, train_acc_value= sess_outputs[1:]
    print('[%s] Iteration %d, train loss = %f, train accuracy = %f' %
        (datetime.now(), step, train_loss_value, train_acc_value))

  # validation
  if step > 0 and step % FLAGS.val_interval == 0:
    print('Evaluating...')
    n_val_samples = 10000
    val_batch_size = FLAGS.val_batch_size
    n_val_batch = int(n_val_samples / val_batch_size)
    val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
    val_labels = np.zeros((n_val_samples), dtype=np.int64)
    val_losses = []
    for i in range(n_val_batch):
      fetches = [logits, label_batch, loss]
      session_outputs = sess.run(
        fetches, {phase_train.name: False})
      val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
      val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
      val_losses.append(session_outputs[2])
    pred_labels = np.argmax(val_logits, axis=1)
    val_accuracy = np.count_nonzero(
      pred_labels == val_labels) / n_val_samples
    val_loss = float(np.mean(np.asarray(val_losses)))
    print('Test accuracy = %f' % val_accuracy)

[2018-07-16 17:20:17.702219] Iteration 100, train loss = 0.184745, train accuracy = 0.976562
[2018-07-16 17:20:23.315353] Iteration 200, train loss = 0.151817, train accuracy = 0.992188
[2018-07-16 17:20:28.926629] Iteration 300, train loss = 0.154841, train accuracy = 0.992188
[2018-07-16 17:20:34.552548] Iteration 400, train loss = 0.172313, train accuracy = 0.984375
[2018-07-16 17:20:40.176353] Iteration 500, train loss = 0.179677, train accuracy = 0.976562
[2018-07-16 17:20:45.813841] Iteration 600, train loss = 0.167317, train accuracy = 0.984375
[2018-07-16 17:20:51.437417] Iteration 700, train loss = 0.136904, train accuracy = 1.000000
[2018-07-16 17:20:57.060942] Iteration 800, train loss = 0.149174, train accuracy = 0.992188
[2018-07-16 17:21:02.688855] Iteration 900, train loss = 0.164953, train accuracy = 0.992188
[2018-07-16 17:21:08.328934] Iteration 1000, train loss = 0.140093, train accuracy = 1.000000
Evaluating...
Test accuracy = 0.917700
[2018-07-16 17:21:15.632367] I

第三轮  量化
prune_rate = 0.85

In [29]:
print('prune前的准确率')
n_val_samples = 10000
val_batch_size = FLAGS.val_batch_size
n_val_batch = int(n_val_samples / val_batch_size)
val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
val_labels = np.zeros((n_val_samples), dtype=np.int64)
val_losses = []
for i in range(n_val_batch):
    fetches = [logits, label_batch, loss]
    session_outputs = sess.run(fetches, {phase_train.name: False})
    val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
    val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
val_losses.append(session_outputs[2])
pred_labels = np.argmax(val_logits, axis=1)
val_accuracy = np.count_nonzero(pred_labels == val_labels) / n_val_samples
val_loss = float(np.mean(np.asarray(val_losses)))
print('Test accuracy = %f' % val_accuracy)

prune前的准确率
Test accuracy = 0.921700


In [30]:
print('prune_rate =0.75时，可视化部分参数：res_net/conv_last/bias:0')
print(sess.run(tf.get_collection('last_biases')[0]))

prune_rate =0.75时，可视化部分参数：res_net/conv_last/bias:0
[ 0.03125    -0.0625      0.00782731  0.125       0.0318545  -0.03125
  0.015625    0.03125     0.01987953 -0.125     ]


In [31]:
para_dict = {}
one_dict = {}
var_name = []
for k in tf.trainable_variables():
    para_dict[k.name] = k
    one_dict[k.name] =tf.ones_like(k)
    var_name.append(k.name)
prune_dict=apply_inq(para_dict,one_dict,var_name,0.85)
trainer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = trainer.compute_gradients(loss)
grads_and_vars = apply_prune_on_grads(grads_and_vars, prune_dict)
train_step = trainer.apply_gradients(grads_and_vars)

In [32]:
for step in range(FLAGS.max_steps):
  if step <= 48000:
    _lr = 1e-2
  else:
    _lr = 1e-3
  if curr_lr != _lr:
    curr_lr = _lr
    print('Learning rate set to %f' % curr_lr)

  # train
  fetches = [train_step, loss]
  if step > 0 and step % FLAGS.summary_interval == 0:
    fetches += [accuracy]
  sess_outputs = sess.run(fetches, {phase_train.name: True, learning_rate.name: curr_lr})


  if step > 0 and step % FLAGS.summary_interval == 0:
    train_loss_value, train_acc_value= sess_outputs[1:]
    print('[%s] Iteration %d, train loss = %f, train accuracy = %f' %
        (datetime.now(), step, train_loss_value, train_acc_value))

  # validation
  if step > 0 and step % FLAGS.val_interval == 0:
    print('Evaluating...')
    n_val_samples = 10000
    val_batch_size = FLAGS.val_batch_size
    n_val_batch = int(n_val_samples / val_batch_size)
    val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
    val_labels = np.zeros((n_val_samples), dtype=np.int64)
    val_losses = []
    for i in range(n_val_batch):
      fetches = [logits, label_batch, loss]
      session_outputs = sess.run(
        fetches, {phase_train.name: False})
      val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
      val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
      val_losses.append(session_outputs[2])
    pred_labels = np.argmax(val_logits, axis=1)
    val_accuracy = np.count_nonzero(
      pred_labels == val_labels) / n_val_samples
    val_loss = float(np.mean(np.asarray(val_losses)))
    print('Test accuracy = %f' % val_accuracy)

Learning rate set to 0.010000
[2018-07-16 18:38:55.357575] Iteration 100, train loss = 0.152917, train accuracy = 1.000000
[2018-07-16 18:39:01.016905] Iteration 200, train loss = 0.142545, train accuracy = 1.000000
[2018-07-16 18:39:06.656253] Iteration 300, train loss = 0.147076, train accuracy = 1.000000
[2018-07-16 18:39:12.300169] Iteration 400, train loss = 0.140761, train accuracy = 1.000000
[2018-07-16 18:39:17.966714] Iteration 500, train loss = 0.159800, train accuracy = 0.992188
[2018-07-16 18:39:23.617357] Iteration 600, train loss = 0.153749, train accuracy = 0.992188
[2018-07-16 18:39:29.257151] Iteration 700, train loss = 0.136645, train accuracy = 1.000000
[2018-07-16 18:39:34.923850] Iteration 800, train loss = 0.129963, train accuracy = 1.000000
[2018-07-16 18:39:40.562526] Iteration 900, train loss = 0.146600, train accuracy = 1.000000
[2018-07-16 18:39:46.236006] Iteration 1000, train loss = 0.148306, train accuracy = 0.992188
Evaluating...
Test accuracy = 0.917800


第四轮  量化  
prune_rate = 1.00

In [33]:
print('prune前的准确率')
n_val_samples = 10000
val_batch_size = FLAGS.val_batch_size
n_val_batch = int(n_val_samples / val_batch_size)
val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
val_labels = np.zeros((n_val_samples), dtype=np.int64)
val_losses = []
for i in range(n_val_batch):
    fetches = [logits, label_batch, loss]
    session_outputs = sess.run(fetches, {phase_train.name: False})
    val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
    val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
val_losses.append(session_outputs[2])
pred_labels = np.argmax(val_logits, axis=1)
val_accuracy = np.count_nonzero(pred_labels == val_labels) / n_val_samples
val_loss = float(np.mean(np.asarray(val_losses)))
print('Test accuracy = %f' % val_accuracy)

prune前的准确率
Test accuracy = 0.922200


In [34]:
print('prune_rate =0.85时，可视化部分参数：res_net/conv_last/bias:0')
print(sess.run(tf.get_collection('last_biases')[0]))

prune_rate =0.85时，可视化部分参数：res_net/conv_last/bias:0
[ 0.03125    -0.0625      0.0361976   0.125       0.03125    -0.03125
  0.01740384  0.03125     0.015625   -0.125     ]


In [35]:
para_dict = {}
one_dict = {}
var_name = []
for k in tf.trainable_variables():
    para_dict[k.name] = k
    one_dict[k.name] =tf.ones_like(k)
    var_name.append(k.name)
prune_dict=apply_inq(para_dict,one_dict,var_name,1)
trainer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = trainer.compute_gradients(loss)
grads_and_vars = apply_prune_on_grads(grads_and_vars, prune_dict)
train_step = trainer.apply_gradients(grads_and_vars)

In [36]:
for step in range(FLAGS.max_steps):
  if step <= 48000:
    _lr = 1e-2
  else:
    _lr = 1e-3
  if curr_lr != _lr:
    curr_lr = _lr
    print('Learning rate set to %f' % curr_lr)

  # train
  fetches = [train_step, loss]
  if step > 0 and step % FLAGS.summary_interval == 0:
    fetches += [accuracy]
  sess_outputs = sess.run(fetches, {phase_train.name: True, learning_rate.name: curr_lr})


  if step > 0 and step % FLAGS.summary_interval == 0:
    train_loss_value, train_acc_value= sess_outputs[1:]
    print('[%s] Iteration %d, train loss = %f, train accuracy = %f' %
        (datetime.now(), step, train_loss_value, train_acc_value))

  # validation
  if step > 0 and step % FLAGS.val_interval == 0:
    print('Evaluating...')
    n_val_samples = 10000
    val_batch_size = FLAGS.val_batch_size
    n_val_batch = int(n_val_samples / val_batch_size)
    val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
    val_labels = np.zeros((n_val_samples), dtype=np.int64)
    val_losses = []
    for i in range(n_val_batch):
      fetches = [logits, label_batch, loss]
      session_outputs = sess.run(
        fetches, {phase_train.name: False})
      val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
      val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
      val_losses.append(session_outputs[2])
    pred_labels = np.argmax(val_logits, axis=1)
    val_accuracy = np.count_nonzero(
      pred_labels == val_labels) / n_val_samples
    val_loss = float(np.mean(np.asarray(val_losses)))
    print('Test accuracy = %f' % val_accuracy)

Learning rate set to 0.010000
[2018-07-16 19:58:16.385923] Iteration 100, train loss = 0.186130, train accuracy = 0.984375
[2018-07-16 19:58:21.138481] Iteration 200, train loss = 0.182097, train accuracy = 0.976562
[2018-07-16 19:58:25.886719] Iteration 300, train loss = 0.199887, train accuracy = 0.968750
[2018-07-16 19:58:30.639712] Iteration 400, train loss = 0.216577, train accuracy = 0.976562
[2018-07-16 19:58:35.396355] Iteration 500, train loss = 0.183710, train accuracy = 0.976562
[2018-07-16 19:58:40.164101] Iteration 600, train loss = 0.139521, train accuracy = 1.000000
[2018-07-16 19:58:44.957159] Iteration 700, train loss = 0.200165, train accuracy = 0.984375
[2018-07-16 19:58:49.706191] Iteration 800, train loss = 0.166233, train accuracy = 0.984375
[2018-07-16 19:58:54.476525] Iteration 900, train loss = 0.193392, train accuracy = 0.976562
[2018-07-16 19:58:59.240757] Iteration 1000, train loss = 0.182412, train accuracy = 0.984375
Evaluating...
Test accuracy = 0.907000


In [37]:
print('prune后的准确率')
n_val_samples = 10000
val_batch_size = FLAGS.val_batch_size
n_val_batch = int(n_val_samples / val_batch_size)
val_logits = np.zeros((n_val_samples, 10), dtype=np.float32)
val_labels = np.zeros((n_val_samples), dtype=np.int64)
val_losses = []
for i in range(n_val_batch):
    fetches = [logits, label_batch, loss]
    session_outputs = sess.run(fetches, {phase_train.name: False})
    val_logits[i*val_batch_size:(i+1)*val_batch_size, :] = session_outputs[0]
    val_labels[i*val_batch_size:(i+1)*val_batch_size] = session_outputs[1]
val_losses.append(session_outputs[2])
pred_labels = np.argmax(val_logits, axis=1)
val_accuracy = np.count_nonzero(pred_labels == val_labels) / n_val_samples
val_loss = float(np.mean(np.asarray(val_losses)))
print('Test accuracy = %f' % val_accuracy)

prune后的准确率
Test accuracy = 0.906900


In [38]:
print('prune_rate =1.00时，可视化部分参数：res_net/conv_last/bias:0')
print(sess.run(tf.get_collection('last_biases')[0]))

prune_rate =1.00时，可视化部分参数：res_net/conv_last/bias:0
[ 0.03125  -0.0625    0.03125   0.125     0.03125  -0.03125   0.015625
  0.03125   0.015625 -0.125   ]
